In [1]:
import numpy as np 
import sys, os
sys.path.append('../Netket/')
import netket as nk
from jax import numpy as jnp
import itertools
from scipy.special import comb
from jax import jit, vmap
import jax
import matplotlib.pyplot as plt 
from cluster_expansion import fwht_coeffs_in_cluster_col_order, prepare_fwht_meta_cached, compress_and_reconstruct_cached, _get_topk_indices_jit

/scratch/ashankar/DataMiningRBMs/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n_sites_test = 4
hilb_test = nk.hilbert.Spin(0.5, n_sites_test)


psi_test = np.random.rand(2**n_sites_test) + 1j * np.random.rand(2**n_sites_test)


cluster_coeffs_test = fwht_coeffs_in_cluster_col_order(np.log(psi_test), hilb_test)

# warm-up / measure compile time for a specific num_kept
k_test = 2**n_sites_test  # you can vary this
# print(f"Preparing cached meta for n_sites={n_sites_test}...", flush=True)
prepare_fwht_meta_cached(hilb_test)  # fill cache (fast)

psi_rec = compress_and_reconstruct_cached(cluster_coeffs_test, k_test, hilb_test)


mask = _get_topk_indices_jit(cluster_coeffs_test, 2) ## correctly returns the mask
coeffs_sel = cluster_coeffs_test[mask] ## correctly retruns bigges coeffs



np.abs(cluster_coeffs_test), np.abs(coeffs_sel), mask, np.isclose(psi_test, psi_rec).all()

(array([1.01698057, 0.14455068, 0.20373374, 0.21530328, 0.13190963,
        0.06156719, 0.43267748, 0.2315019 , 0.05741601, 0.22965086,
        0.10069121, 0.19031212, 0.03328813, 0.17182   , 0.2036511 ,
        0.22495747]),
 array([1.01698057, 0.43267748]),
 Array([0, 6], dtype=int32),
 np.True_)